# Setup

In [ ]:
!pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 922.9 kB/s eta 0:00:00


In [ ]:
import re
import nltk
import spacy
import copy
import openai
import numpy as np

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk import Tree
from nltk.stem import *
from nltk.stem.porter import *

from spacy.attrs import ORTH, NORM
from spacy import displacy
from spacy.tokens import Doc
from spacy.language import Language
from spacy.symbols import ORTH
from spacy.tokenizer import Tokenizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
nlp = spacy.load("en_core_web_sm")

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


# Pre-processing

## GPT prompt (from input text)

input text

In [ ]:
text = """
A company operates four departments. Each department employs employees. Each of the employees may or may not have one or more dependents. Each employee may or may not have an employment history.
"""

Correct results of entities and attributes from true data (used for testing purposes to compute precision, recall and F1 score)

In [ ]:
true_res = {'member': ['memdid', 'name', 'zip', 'date'], 'membership type': ['mname', 'price', 'mid'], 'sale transaction': ['date', 'tid'], 'merchandise item': ['mrchid', 'price', 'name'], 'day pass': ['ID', 'date'], 'pass category': ['passcatid', 'pcname', 'price']}
true_card = [['member', 'membership type'], ['memberships type', 'member'], ['member', 'day pass'], ['day pass', 'member'], ['day pass', 'pass category'], ['pass category', 'day pass'], ['member', 'sale transaction'], ['sale transaction', 'member'], ['sale transaction', 'merchandise item'], ['merchandise item', 'sale transaction']]
true_unique = [[1, 'member','memdid'], [1,'membership type','mid'], [1,'day pass','ID'], [1,'pass category','passcatid'], [1,'sale transaction','tid'],[1,'merchandise item','mrchid']]

openai key

In [ ]:
# Key that gives an access to openai account
openai.api_key = 'key_code'

GPT 4.0 examples for few-shot prompting for text simplification

In [ ]:
# ex12
ex1_input = """
The company is organised into departments. Each department has a unique name, a unique number, and a particular employee who manages and/or controls the department.
We keep track of the start date when that employee began managing the department. A department may have several locations.
A department controls a number of projects, each of which has a unique name, a unique number, and a single location.
We store each employee’s name, Social Security number, address, salary, sex (gender), birth date and multiple phone numbers.
An employee is assigned to one department, but may work on several projects, which are not necessarily controlled by the same department.
We keep track of the current number of hours per week that an employee works on each project. We also keep track of the direct supervisor of each employee (who is another employee).
We want to keep track of the dependents of each employee for insurance purposes. We keep each dependent’s first name, sex, number, birth date, and relationship to the employee.
"""

ex1_output = """
Each department has a name and number. The department’s name is unique. The department’s number is unique.
The department is managed by a unique employee. The managing has a starting date. A department may have several locations.
A department controls zero or more projects. The project has a name, number, and location. The project’s name is unique.
The project’s number is unique. The employee has a Social Security Number, address, salary, sex, birth date and phone numbers.
An employee is assigned to one department. An employee can work on several projects. The working has a current number of hours per week.
An employee can supervise zero or many employees. An employee can have one or more dependents. A dependent has a first name, sex, number, birth date, and relationship.
"""

# ex11
ex2_input = """
The database must store painters, paintings, and galleries information. Painters have a unique number, Name, and phone number.
Paintings have a unique number, several colors, title and price. Galleries have a unique number, owner, phone number, commission rate and address.
A painting is painted and created by a particular artist, and that painting is exhibited in a particular gallery.
A gallery can exhibit at least one paintings, but each painting can be exhibited in only one gallery.
Similarity, a painting is painted by a single painter, but each painter can paint many paintings.
"""

ex2_output = """
The database must store painters, paintings, and galleries.
Painters have a number, name, and phone number. The painter’s number is unique. Paintings have a number, colors, title and price.
The painting’s number is unique. Galleries have a number, owner, phone number, commission rate and address.
The gallerie’s number is unique. A painting is painted by a unique artist, and the painting is exhibited in a unique gallery.
A gallery can exhibit one or more paintings, but each painting can be exhibited in only one gallery.
A painting is painted by a single painter, but each painter can paint many paintings.
"""

GPT prompt to simplify text in order to easier extract entities, attributes and relationships from complex english

In [ ]:
# parameter controlling randomness of the output
temp = 0.0

def get_completion(prompt, model="gpt-4"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temp, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

Prompt

In [ ]:
requirements = f"""
{text}
"""

prompt = f"""
The task is to extract an Entity-relationship diagram from natural language requirements.
To do that, these requirements should be simplified as much as possible for the extraction of entities,
attributes and relationships (i.e. short and simple sentences, explicit subjects, etc.).
Two examples of semplification are given below, where the 'input text' is the original text and the 'output text' is the simplified text:

Input text:
'
{ex1_input}
'

Output text:
'
{ex1_output}
'

Input text:
'
{ex2_input}
'

Output text:
'
{ex2_output}
'

Simplify as shown above the following text delimited by triple backticks, remembering to not remove any information, creating short and simple sentences and pay attention to not repeat any sentence if there are synonyms.
If there are words composed with '-' or '_' or in camel case, maintain only the main token (i.e. the possessed token, e.g. in 'person_id', maintain id).
If in the sentence there is the possibility to choose among two verbs (e.g. 'a painter paints and/or creates many paintings'), maintaing only the verb used previously or after this sentence (in the case of 'a painter paints and/or creates many painting.
A painting is painted by one painter', maintain only the verb 'paints', i.e. 'a painter paints many painting. A painting is painted by one painter').
Return as output only the modified text without backticks or additional spaces or text:

Text:
```{requirements}```
"""

In [ ]:
def gpt_output():
  answers = []
  for i in range(1):
      response = get_completion(prompt)
      answers.append(response)

  return answers[0]

Clean GPT output (remotion of additional spaces, backticks, etc.)

In [ ]:
def clean_gpt_output(text):
  new_text = ''
  i = 0
  while i < len(text):
    if i == '`' and i + 2 < len(text) and text[i+1] == '`' and text[i+2] == '`':
      i += 3
    elif (text[i] == '\'' and i+1 < len(text) and text[i+1] == '\n') or  (text[i] == '\n' and i+1 < len(text) and text[i+1] == '\n') or (text[i] == '\n' and i+1 < len(text) and text[i+1] == '\''):
      i += 3
    else:
      new_text += text[i]
      i += 1

  return new_text

## Text pre-processing

Sentence segmentation

First three methods not used since last one is the best one (for comparation purposes)

In [ ]:
# -------------------- SENTENCE SEGMENTATION --------------------

# write them lower and compare lower text with them -> etc?
abbreviations = [ 'a.', 'b.', 'c.', 'd.', 'e.', 'f.', 'g.', 'h.' 'k', 'j.', 'i.', 'l.',
                 'm.', 'n.', 'o.', 'p.', 'q.', 'r.', 's.', 't.', 'u.', 'v.', 'w.', 'x.', 'y.', 'z.', 'mr.', 'ms.', 'mrs.', 'dr.', 'prof.',
                 'capt.', 'rev.', 'col.', 'sgt.', 'st.', 'u.s.', 'i.e.', 'e.g.', 'ca.', 'i.a.', 'no.', 'p.s.', 'aka.', 'fig.',
                 'cit.', 'v.', 'vol.']

# if period, space and uppercase or some variations then split
def sentence_segmentation_one(text):
    finalSegmentation = []
    mergedParts = ""
    for i in range(0, len(text)-2):
        if ((text[i] == '.' and text[i+1] == ' ' and text[i+2].isupper())
        or (i <= len(text)-4 and text[i] == '.' and text[i+1] == '"'
            and text[i+2] == ' ' and text[i+1].isupper())
        or (i <= len(text - 4) and text[i] == '.' and text[i+1] == ' '
            and text[i+2] == '"' and text[i+3].isupper())
        or (i > len(text)-3 and text[i] == '.')):
            finalSegmentation.append(mergedParts)
            mergedParts = ""
        else:
            mergedParts += text[i]
    return finalSegmentation

# if same as above but previous word is not an abbreviation, then split
def sentence_segmentation_two(text):
    finalSegmentation = []
    mergedParts = ""
    temp = ""
    for i in range(0, len(text)-2):
        if (((text[i] == '.' and text[i+1] == ' ' and text[i+2].isupper())
        or (i <= len(text)-4 and text[i] == '.' and text[i+1] == '"'
            and text[i+2] == ' ' and text[i+1].isupper())
        or (i <= len(text - 4) and text[i] == '.' and text[i+1] == ' '
            and text[i+2] == '"' and text[i+3].isupper())
        or (i > len(text)-3 and text[i] == '.'))
        and (temp.split()[-1].lower() not in abbreviations)):
            finalSegmentation.append(mergedParts)
            mergedParts = ""
        else:
            mergedParts += text[i]
            temp += text[i]
    return finalSegmentation

def sentence_segmentation_three(text):
    pattern = r'(?<=[.!?]) +(?=[A-Z])'
    finalSegmentation = re.split(pattern, text)
    return finalSegmentation



# implementation of sent_tokenize + join sentences if abbreviations that doesn't define end of sentence
def sentence_segmentation_mine(text):
    # general sentence segmentation
    firstSegmentation = sent_tokenize(text)
    print(firstSegmentation)
    finalSegmentation = []
    i = 0
    while i < len(firstSegmentation):
    # for i in range(0, len(firstSegmentation)):
      finalToken = firstSegmentation[i].split()[-1]
      # if abbreviation at the end of a sentence split by nltk, merge current sentence with next sentence
      if finalToken.lower() in abbreviations and i < len(firstSegmentation)-1:
        segment = firstSegmentation[i] + ' ' + firstSegmentation[i+1]
        # check for next sentences to merge (all subsequent sentences which ends with an abbreviation)
        j = i+1
        while j < len(firstSegmentation):
          finalToken = firstSegmentation[j].split()[-1]
          if finalToken.lower() in abbreviations:
            segment += ' ' + firstSegmentation[j]
            j += 1
          # if no more sentences that end with abbreviation, store all merged sentences and restart from current last sentence
          else:
            finalSegmentation.append(segment)
            i = j + 1
            break
      else:
        finalSegmentation.append(firstSegmentation[i])
        i += 1
    return finalSegmentation

Lower-case text transformation

In [ ]:
# id in lowercase is not recognise as a single token by spaCy, thus transform it in uppercase (recognised) if encountered

def replace_word_with_uppercase(sentence, word):
    # Define a regular expression pattern to match the word
    pattern = re.compile(r'\b{}\b'.format(re.escape(word)))
    # Replace the matched word with its uppercase version
    replaced_sentence = pattern.sub(word.upper(), sentence)
    return replaced_sentence


def to_lowercase(sentences):
  lowerText = []
  for sentence in sentences:
    lowerText.append(sentence.lower())
  for i in range(0, len(lowerText)):
    if 'id' in lowerText[i]:
      lowerText[i] = replace_word_with_uppercase(lowerText[i], 'id')
  return lowerText




Pronoun substitution

Find nouns in the text, find pronouns and substitute them with antecedent subject

In [ ]:
# find all subjects in the text with the position of the first token

subjs_noun = []

def subject_extraction_for_pron(sentences):
    for sentence in sentences:
      temp_subj_nouns = []
      doc = nlp(sentence)
      i = 0
      while i < len(doc):
        # last condition works for composite words as 'booking dates'
        if doc[i].pos_ == 'NOUN' or doc[i].pos_ == 'PROPN' or (doc[i].pos_ == 'VERB' and i+1 < len(doc) and doc[i].dep_ == 'compound'):
          sentence = str(doc[i])
          # save last token of noun info to know if it is a subject or not
          subj_info = doc[i].dep_
          # check if special case of 'first name', 'middle name', 'last name', 'serial number'
          if i-1 >= 0 and (str(doc[i]) == 'name' and (str(doc[i-1]) == 'first' or str(doc[i-1]) == 'last' or str(doc[i-1]) == 'middle') or (str(doc[i-1]) == 'serial' and str(doc[i]) == 'number')):
            sentence = str(doc[i-1]) + ' ' + str(doc[i])
          # if noun as last token of the sentence insert in temp_nouns and insert temp_nouns to nouns at the end of the code and continue with next sentence
          if i == len(doc) - 1 and (subj_info == 'nsubj' or subj_info == 'nsubjpass' or subj_info == 'csubj' or subj_info == 'csubjpass'):
            temp_subj_nouns.append([sentence, i])
            break
          j = i + 1
          while j < len(doc):
            # handle manually problem of tags of 'ID' -> if compound with ID, first token doesn't have the compound tag, thus check only if next token is ID
            if doc[j-1].dep_ == 'compound' and (doc[j].pos_ == 'NOUN' or doc[j].pos_ == 'PROPN' or doc[j].pos_ == 'VERB'):
              sentence += ' ' + str(doc[j])
              subj_info = doc[j].dep_
              j+= 1
            else:
              if subj_info == 'nsubj' or subj_info == 'nsubjpass' or subj_info == 'csubj' or subj_info == 'csubjpass':
                temp_subj_nouns.append([sentence, i])
              i = j - 1
              break
        i+=1
      subjs_noun.append(temp_subj_nouns)
    return subjs_noun

p_pronouns = ['i', 'me', 'she', 'her', 'he', 'him', 'you', 'they', 'them', 'we', 'us', 'it']

def pronoun_substitution(sentences):
  no_pronoun_text = []
  # start from second sentence and check if there are some pronouns. If yes, get last subject of previous sentence and substitute pronoun with that subject
  for i in range(0, len(sentences)):
    temp_sentence = ""
    doc = nlp(sentences[i])
    j = 0
    while j < len(doc):
      if doc[j].pos_ == 'PRON' and str(doc[j]).lower() in p_pronouns:
        # pick last subject from last sentence only if there is not a subject in the current sentence before the pronoun (pick previous closest subject)
        # e.g. 'Anita loves eating and she is thin.'
        if len(subjs_noun[i]) > 0:
          prev_subj = ''
          prev_subj_pos = -1
          for elem in subjs_noun[i]:
            if elem[1] < j and elem[1] > prev_subj_pos:
              prev_subj = elem[0]
              prev_subj_pos = elem[1]
          # if previous subject found in current sentence, substitute, otherwise pick last subject in previous sentence
          if prev_subj_pos != -1:
            # if pronoun is at the beginning of the sentence, don't add a space before, otherwise yes
            if j == 0:
              temp_sentence = prev_subj
            else:
              temp_sentence += ' ' + prev_subj
          else:
            if i > 0:
              prev_subj = subjs_noun[i-1][-1][0]
              if j == 0:
                temp_sentence = prev_subj
              else:
                temp_sentence += ' ' + prev_subj
      else:
        if j == 0:
          temp_sentence = str(doc[j])
        else:
          temp_sentence += ' ' + str(doc[j])
      j += 1

    no_pronoun_text.append(temp_sentence)
    temp_sentence = []
  return no_pronoun_text


Text lemmatization

In [ ]:
def text_lemmatization(sentences):
  lem_text = []
  for sentence in sentences:
    doc = nlp(sentence)
    lem_sentence = ' '.join([token.lemma_ for token in doc])
    lem_text.append(lem_sentence)
  return lem_text

Handmade list of common attribute

In [ ]:
# taken from internet sources.
# id in uppercase since we put it manually in this way
common_attr = [
    'name', 'last name', 'first name', 'middle', 'description', 'address', 'sex', 'ID', 'number',
    'no', 'code', 'date', 'version', 'status', 'volume', 'birth', 'title', 'price',
    'amount', 'value', 'phone number', 'isbn', 'birth date', 'enable', 'disable', 'label', 'title', 'size',
    'length', 'width', 'height', 'weight', 'color', 'quantity', 'time', 'timestamp',
    'start date', 'starting date', 'ending date', 'end date', 'start time', 'end time', 'due date', 'priority',
    'latitude', 'longitude', 'phone', 'fax', 'url', 'comment', 'note', 'result', 'option', 'setting',
    'permission', 'configuration', 'authentication', 'constraint', 'requirement', 'condition',
    'date of birth', 'serial number', 'zip code', 'feedback', 'homepage url', 'webpage url', 'starting year',
    'ending year', 'start year', 'end year'
    ]

# Extraction methods - Modeling part

## Noun extraction

In [ ]:
# -------------------- NOUNS EXTRACTION - BASIC --------------------

# format -> [noun, starting position in sentence, nr of tokens which compose the noun, .pos_ of last token, .dep_ of last token, [array of .ent_type_ of each token in order], token.tag_]:
  # 1. the string representing the noun (monogram, bigram, etc)
  # 2. to easily retrieve the noun, store position of first token of the noun
  # 3. to know how many tokens to jump to continue the sentence, store nr of tokens which compose the nouns
  # 4. .pos_ = DET, NOUN, ADJ to know role of noun
  # 5. .dep_ = nsubjpass, amod, det, etc., to know other information about the token (mainly to detect subjects of sentences)
  # 6. .ent_type_ = PERSON, GPE, to remove proper names of people and locations
  # 7. .tag_ = to identify if the noun is singular or plural (mainly for recognition of relationships based on possession)

# triple array which separe nouns for each sentence => s1 = [[info1], [info2], ...].

def nouns_extraction(sentences):
  nouns = []
  for sentence in sentences:
    temp_nouns = []
    doc = nlp(sentence)
    # print(doc)
    i = 0
    while i < len(doc):
      composed_of = False
      # last condition works for composite words as booking dates
      if doc[i].pos_ == 'NOUN' or doc[i].pos_ == 'PROPN' or ((doc[i].pos_ == 'VERB' or doc[i].pos_ == 'ADJ') and i+1 < len(doc) and doc[i].dep_ == 'compound'):
        sentence = str(doc[i])
        # print('doc[i]:', sentence, doc[i].pos_)

        ent_type = [doc[i].ent_type_]
        last_info = [sentence, i, 1, doc[i].pos_, doc[i].dep_, ent_type, doc[i].tag_]

        # handle special case in which composite noun has 'of' -> level of difficulty, etc.
        if i + 1 < len(doc) and str(doc[i+1]) == 'of' and (str(doc[i]) == 'level' or str(doc[i]) == 'degree' or str(doc[i]) == 'quality' or str(doc[i]) == 'state' or str(doc[i]) == 'form' or str(doc[i]) == 'number'):
          last_info[0] = str(doc[i]) + ' ' + str(doc[i+1])
          last_info[1] = i
          last_info[2] = 2
          last_info[5] = [doc[i+1].ent_type_] + last_info[5]
          i += 1
          composed_of = True
        # check if special case of 'first name' or 'last name' or 'serial number'
        if i-1 >= 0 and ((str(doc[i]) == 'name' and (str(doc[i-1]) == 'first' or str(doc[i-1]) == 'last' or str(doc[i-1]) == 'middle')) or (str(doc[i-1]) == 'serial' and str(doc[i]) == 'number') or (str(doc[i-1]) == 'starting' and str(doc[i]) == 'date') or (str(doc[i-1]) == 'ending' and str(doc[i]) == 'date')):
          last_info[0] = str(doc[i-1]) + ' ' + str(doc[i])
          last_info[1] = i-1
          last_info[2] = 2
          last_info[5] = [doc[i-1].ent_type_] + last_info[5]
        # if noun as last token of the sentence insert in temp_nouns and insert temp_nouns to nouns at the end of the code and continue with next sentence
        if i == len(doc) - 1:
          temp_nouns.append(last_info)
          break

        j = i + 1

        while j < len(doc):
          # print('doc[j]:', doc[j])
          if (doc[j-1].dep_ == 'compound' and (doc[j].pos_ == 'NOUN' or doc[j].pos_ == 'PROPN' or doc[j].pos_ == 'VERB')) or str(doc[j]) == 'ID' or (composed_of and (doc[j].pos_ == 'NOUN' or doc[j].pos_ == 'PROPN')) or ((doc[j-1].pos_ == 'NOUN' or doc[j-1].pos_ == 'PROPN') and (doc[j].pos_ == 'NOUN' or doc[j].pos_ == 'PROPN')):
            last_info[0] += ' ' + str(doc[j])
            last_info[2] += 1
            last_info[3] = doc[j].pos_
            last_info[4] = doc[j].dep_
            last_info[5].append(doc[j].ent_type_)
            last_info[6] = doc[j].tag_

            j+= 1
          elif str(doc[j]) == '-' and j+1 < len(doc) and (doc[j+1].pos_ == 'NOUN' or doc[j+1].pos_ == 'PROPN'):
            last_info[0] += str(doc[j]) + str(doc[j+1])
            last_info[2] += 2
            last_info[3] = doc[j+1].pos_
            last_info[4] = doc[j+1].dep_
            last_info[5].append(doc[j].ent_type_)
            last_info[5].append(doc[j+1].ent_type_)
            last_info[6] = doc[j+1].tag_
            j += 2
          else:
            i = j - 1
            # if  not any(word in last_info[0] for word in db_nouns):
            temp_nouns.append(last_info)
            composed_of = False
            break
      composed_of = False
      i+=1
    nouns.append(temp_nouns)
  return nouns

People and locations proper names removal from noun array

In [ ]:
# remove from noun list -> use ent_type tag, if 'PERSON' or 'GPE' then remove. Remove only if entire noun is made of these two tags

person_location_tags = ['PERSON', 'GPE']
# specific nouns we need to maintain which tags are PERSON or GPE
maintained_nouns = ['db']

def proper_noun_removal(nouns_array):
  new_nouns_array = []
  for i in range(0, len(nouns_array)):
    sentence_nouns = []
    for j in range(0, len(nouns_array[i])):
      nn = nouns_array[i][j].copy()
      nr_tags = len(nouns_array[i][j][5])
      remove = 0
      if nr_tags > 0:
        for tag in nouns_array[i][j][5]:
          if tag == 'PERSON' or tag == 'GPE':
            remove += 1
        if remove < nr_tags or ((len(nouns_array[i][j]) > 0 and len(nouns_array[i][j][0].split('_')) > 1 and nouns_array[i][j][0].split('_')[-1].lower() == 'id') or nouns_array[i][j][0] in maintained_nouns):
          sentence_nouns.append(nn)
    new_nouns_array.append(sentence_nouns)
    sentence_nouns = []
  return new_nouns_array

Noun lemmatization

In [ ]:
def noun_lemmatization(nouns_array):
  lemmatized_nouns = []
  for i in range(0, len(nouns)):
    temp = []
    for j in range(0, len(nouns[i])):
      noun = nouns[i][j].copy()
      doc_nn = nlp(noun[0])
      lem_noun = ' '.join([token.lemma_ for token in doc_nn])
      noun[0] = lem_noun
      temp.append(noun)
    lemmatized_nouns.append(temp)
    temp = []
  return lemmatized_nouns

## Entity type/attribute extraction

Extract entity types and attributes based on the heuristic rule of the underscore (or dash).

If entities and attributes in the form of ent_attr or ent-attr, e.g. student_id or student-id, remove first part which identifies the entity and associated with it the second part which identifies the attribute -> check first if it's not an attribute only (e.g. phone_number)

In [ ]:
def extract_underscore(nouns, ent_attr):
  for i in range(0, len(nouns)):
    for j in range(0, len(nouns[i])):
      noun = nouns[i][j]
      if '_' in noun[0] or '-' in noun[0]:
        split_noun = noun[0].split('_') if '_' in noun[0] else noun[0].split('-')
        for z in range(0, len(split_noun)):
          split_noun[z] = split_noun[z].strip()
        # check if not completely an attribute
        noun_str = ' '.join(split_noun)
        if noun_str in common_attr:
          continue
        else:
          # pick entity and attribute
          entity = split_noun[0]
          attribute = split_noun[1]
          # put it in uppercase to be consistent
          if attribute == 'id':
            attribute = 'ID'
          doc_entity = nlp(entity)
          entity = ' '.join([token.lemma_ for token in doc_entity])
          if entity not in ent_attr.keys():
            ent_attr[entity] = []
            ent_attr[entity].append(attribute)
          else:
            ent_attr[entity].append(attribute)
  return ent_attr

Extract entity types and attributes from composite nouns

If the second token or more of a composite noun is in the list of common attributes, insert first token as entitx type and following tokens and attributes

In [ ]:
# remove one or more consecutive words (token/s which compose the attribute) from a string (the noun)
def remove_consecutive_words(sentence, words):
    pattern = r'\b(?:' + '|'.join(re.escape(word) for word in words) + r')\b(?:\s+\b(?:' + '|'.join(re.escape(word) for word in words) + r')\b)*'
    return re.sub(pattern, '', sentence)

def extract_common(nouns, ent_attr):
  for sentence in nouns:
    for noun in sentence:
      # there can be more correspondences in the common attributes, e.g. student phone number can correspond with 'number' and 'phone' and 'phone number', then the longest one win
      equal_count = 0
      winning_attr = ''
      # check if noun is composite
      if len(noun[0].split(' ')) > 1:
        for attribute in common_attr:
          if attribute in noun[0]:
            if len(attribute.split(' ')) > equal_count:
              equal_count = len(attribute.split(' '))
              winning_attr = attribute
        if equal_count != 0:
          words_to_remove = winning_attr.split(' ')
          noun_without_attr = remove_consecutive_words(noun[0], words_to_remove)
          # avoid special case in which there is 'isbn value' where value is not important and should not be taken into consideration
          # case in which the noun is empty too
          if noun_without_attr != '' and noun_without_attr == 'isbn' and winning_attr == 'value':
            if noun_without_attr in ent_attr.keys():
              # don't add if attribute already in the list
              if not (winning_attr in ent_attr[noun_without_attr]):
                  ent_attr[noun_without_attr].append(winning_attr)
            else:
              ent_attr[noun_without_attr] = [winning_attr]
  return ent_attr

Extract entities types based on the heuristic rule of subjects

In [ ]:
def subj_extraction(lemmatized_nouns):
  subjects = []
  for sentence in lemmatized_nouns:
    temp_nouns = []
    for noun in sentence:
      if noun[4] == 'nsubj' or noun[4] == 'nsubjpass' or noun[4] == 'csubj' or noun[4] == 'csubjpass':
        temp_nouns.append([noun[0], noun[-1], noun[1]])
    subjects.append(temp_nouns)
    temp_nouns = []
  return subjects

Extract entity type/entity type relationship or entitytype /attribute relationship based on quantity tokens

In [ ]:
# store cardinalities with the two entities or entity/attribute:
# 1. [(1, 01),ent1, ent2] -> one-to-one (zero included)
# 2. [(1, 1),ent1, ent2] -> one-to-one
# 4. [(1, 02),ent1, ent2] -> one-to-many (zero included)
# 3. [(1, 2),ent1, ent2] -> one-to-many

# common quantity tokens
quantity_adv = ['zero', 'one', 'unique', 'single', 'specific', 'sole', 'many', 'more', 'multiple', 'several', 'numerous', 'various', 'two']

def quantity_extr(sentences, subjects, ent_attr, cardinality_attr, cardinality):
  for i in range(0, len(sentences)):
    doc = nlp(sentences[i])
    # iterate over all subjects in the current sentence.
    # starting from the subject, check if after it there is a verb, a quantity and other nouns. If yes, store the other nouns as entities
    j = 0
    while j < len(subjects[i]):
      verb_found = False
      quantity_found = False
      noun_found = False
      quantity = ''
      snd_quantity = ''
      other_subj = ''
      verb = ''
      attr_found = False
      # if a new subject encountered, restart analysis (everything above)
      restart = False
      # look everything after subject -> position of first token of subject + its length + 1 to have next token
      k = subjects[i][j][2] + len(subjects[i][j][0].split(' '))

      while k < len(doc):
        if restart:
          break
        # if encounter another subject, break loop and restart
        elif (j + 1 < len(subjects[i]) and k == subjects[i][j+1][2]):
          restart = True
          verb_found = False
          verb = ''
          quantity_found = False
          quantity = ''
          snd_quantity = ''
          other_subj = ''
          break

        # check if verb found
        elif doc[k].pos_ == 'VERB' or (doc[k].pos_ == 'AUX' and str(doc[k]) == 'be') or (doc[k].dep_ == 'prt' and verb != ''):
          verb_found = True
          # store verb with relationship
          if verb == '':
            verb += str(doc[k])
          else:
            verb += ' ' + str(doc[k])

        # check for quantity token and store it. Store only once (first time)
        elif verb_found and str(doc[k]) in quantity_adv and not quantity_found:
          quantity_found = True
          quantity = str(doc[k])

        # if previous quantity stored is 0, often associated with another quantity (zero or many, zero or one, thus store second quantity)
        elif (quantity == 'zero' or quantity == 'one') and str(doc[k]) in quantity_adv:
          snd_quantity = str(doc[k])

        # if verb and quantity found, check for noun/s. compare current j position with all j positions in nouns array for this specific sentence.
        elif verb_found and quantity_found and (doc[k].pos_ == 'NOUN' or doc[k].pos_ == 'PROPN' or str(doc[k]) == 'last' or str(doc[k]) == 'first' or str(doc[k]) == 'middle' or str(doc[k]) == 'serial'):
          # seach in non-lemmatized nouns since the text is not lemmatized and nouns can be not found
          for noun in nouns[i]:
            if noun[1] == k:
              other_subj = noun[0]
              break
          noun_found = True

          k += len(other_subj.split(' ')) - 1
          # check relationship cardinality -> from position 0 to 3 is one, otherwise multiple. First subject in general considered singular
          # lemmatize noun
          lem = nlp(other_subj)
          lem_token = ' '.join([token.lemma_ for token in lem])
          other_subj = lem_token

          if other_subj not in common_attr:
            if quantity == 'zero' and len(snd_quantity) > 0:
              # zero or one
              if snd_quantity == 'one':
                  cardinality.append([('1','01'), subjects[i][j][0], other_subj, verb])
              # zero or more
              elif snd_quantity in quantity_adv[6:]:
                cardinality.append([('1','02'), subjects[i][j][0], other_subj, verb])
            # one or more
            elif quantity == 'one' and len(snd_quantity) > 0 and snd_quantity in quantity_adv[5:]:
              cardinality.append([('1','12'), subjects[i][j][0], other_subj, verb])
            # one, many, several, more, etc.
            elif snd_quantity == '':
              # one-to-one relationship -> a unique, a sole, a single, one
              if quantity in quantity_adv[1:6]:
                cardinality.append([('1','1'), subjects[i][j][0], other_subj, verb])
              else:
                # one-to-many relationship
                cardinality.append([('1','2'), subjects[i][j][0], other_subj, verb])
          else:
            # other_subj = winning_attr
            if subjects[i][j][0] not in ent_attr.keys():
                ent_attr[subjects[i][j][0]] = [other_subj]
            else:
              if other_subj not in ent_attr[subjects[i][j][0]]:
                ent_attr[subjects[i][j][0]].append(other_subj)

            # one, many, several, more, etc.
            if snd_quantity == '':
              # one-to-one relationship -> a unique, a sole, a single, one
              if quantity in quantity_adv[1:5]:
                if [1, subjects[i][j][0], other_subj] not in cardinality_attr:
                  cardinality_attr.append([1, subjects[i][j][0], other_subj])
              else:
                # one-to-many relationship
                if [2, subjects[i][j][0], other_subj] not in cardinality_attr:
                  cardinality_attr.append([2, subjects[i][j][0], other_subj])

          # now check if there are other entities associated (e.g. 'A professor can have more students but a single course.', 'A professor have a single course and department.')
          # stop if subject found
          attr_found = False
          if noun_found:
            other_subj = ''
            z = k + 1
            quantity_found = False
            while z < len(doc):
              # if encounter another subject, break loop and restart
              if j + 1 < len(subjects[i]) and z == subjects[i][j+1][2]:
                restart = True
                quantity_found = False
                quantity = ''
                snd_quantity = ''
                other_subj = ''
                noun_found = False
                break

              elif str(doc[z]) in quantity_adv:
                quantity_found = True
                quantity = str(doc[z])

              elif quantity == 'zero' and str(doc[z]) in quantity_adv:
                snd_quantity = str(doc[z])

              elif str(doc[z]) == 'and' or str(doc[z]) == ',':
                z += 1
                continue

              elif doc[z].pos_ == 'NOUN' or doc[z].pos_ == 'PROPN' or str(doc[z]) == 'last' or str(doc[z]) == 'first' or str(doc[z]) == 'middle' or str(doc[z]) == 'serial':
                # seach in non-lemmatized nouns since the text is not lemmatized and nouns can be not found
                for noun in nouns[i]:
                  if noun[1] == z:
                    other_subj = noun[0]
                    break

                if other_subj != '':
                  # lemmatize noun
                  lem = nlp(other_subj)
                  lem_token = ' '.join([token.lemma_ for token in lem])
                  other_subj = lem_token

                  if other_subj not in common_attr:
                    if quantity == 'zero' and len(snd_quantity) > 0:
                      # zero or one
                      if snd_quantity == 'one':
                          cardinality.append([('1','01'), subjects[i][j][0], other_subj, verb])
                      # zero or more
                      elif snd_quantity in quantity_adv[6:]:
                        cardinality.append([('1','02'), subjects[i][j][0], other_subj, verb])
                    # one or more
                    elif quantity == 'one' and len(snd_quantity) > 0 and snd_quantity in quantity_adv[5:]:
                      cardinality.append([('1','12'), subjects[i][j][0], other_subj, verb])
                    # one, many, several, more, etc.
                    elif snd_quantity == '':
                      # one-to-one relationship -> a unique, a sole, a single, one
                      if quantity in quantity_adv[1:6]:
                        cardinality.append([('1','1'), subjects[i][j][0], other_subj, verb])
                      else:
                        # one-to-many relationship
                        cardinality.append([('1','2'), subjects[i][j][0], other_subj, verb])
                  else:
                    # other_subj = winning_attr
                    if subjects[i][j][0] not in ent_attr.keys():
                        ent_attr[subjects[i][j][0]] = [other_subj]
                    else:
                      if other_subj not in ent_attr[subjects[i][j][0]]:
                        ent_attr[subjects[i][j][0]].append(other_subj)

                    # one, many, several, more, etc.
                    if snd_quantity == '':
                      # one-to-one relationship -> a unique, a sole, a single, one
                      if quantity in quantity_adv[1:6]:
                        if [1, subjects[i][j][0], other_subj] not in cardinality_attr:
                          cardinality_attr.append([1, subjects[i][j][0], other_subj])
                      else:
                        # one-to-many relationship
                        if [2, subjects[i][j][0], other_subj] not in cardinality_attr:
                          cardinality_attr.append([2, subjects[i][j][0], other_subj])
                  z += len(other_subj.split(' ')) - 1
                  other_subj = ''
                  winning_attr = ''
              z += 1
              k += z
        k += 1
      j += 1
  return ent_attr, cardinality_attr, cardinality

Extract attributes in sentence structure:
 - attribute's noun + 'be' + 'unique'

e.g. 'The student's id is unique.'

In [ ]:
def unique_extr(sentences, cardinality_attr):
  for i in range(0, len(sentences)):
    doc = nlp(sentences[i])
    # iterate over all tokens in the sentence and find a noun
    j = 0
    while j < len(doc):
      for k in range(0, len(nouns[i])):
        # find noun based on position
        if nouns[i][k][1] == j:
          # skip noun's length
          j += len(nouns[i][k][0].split(' '))
          # check for 's and go ahead
          if j < len(doc):
            if doc[j].dep_ == 'case':
              j += 1
              attr = ''
              # find next noun which should be the attribute
              for n in range(0, len(nouns[i])):
                if nouns[i][n][1] == j:
                  attr = nouns[i][n][0]
                  j += len(nouns[i][n][0].split(' '))
                  break
              # check for 'be' verb + unique to have 'entity's attribute is unique'
              if attr != '' and j + 1 < len(doc) and doc[j].pos_ == 'AUX' and str(doc[j+1]) == 'unique':
                nn = nlp(nouns[i][k][0])
                lem_token = ' '.join([token.lemma_ for token in nn])
                cardinality_attr.append([1, lem_token, attr])
                attr = ''
            break
      j += 1
  return cardinality_attr

Extract entities and attributes based on heuristic rule of frequency.

Nouns which are present only once in the nouns list or have frequency (among nouns) below 2% are likely to be attributes, otherwise entity types

In [ ]:
def freq_extraction(nouns):
  # list (one dimension) with all nouns
  all_nouns = []
  for sentence in nouns:
    for noun in sentence:
      all_nouns.append(noun[0])

  tot_nouns = len(all_nouns)

  # dictionary with noun as key and occurences,frequency as values
  noun_freq = {}
  for i in range(0, len(all_nouns)):
    if all_nouns[i] in noun_freq:
      noun_freq[all_nouns[i]][0] += 1
    else:
      noun_freq[all_nouns[i]] = [1,0]

  # add frequency at the end
  for key, value in noun_freq.items():
    value[1] = value[0]/tot_nouns
  # Extract attributes if they occur only once or they frequency is less than 2%
  freq_attr = []
  freq_ent = []

  for key, value in noun_freq.items():
    if value[0] > 1 and value[1] >= 0.02:
      freq_ent.append(key)
    else:
      freq_attr.append(key)
  return freq_ent, freq_attr

Extract entities and attributes based on three heuristic rules:
1. Possessive -> car's serial number - cars' serial number -> car as entity and serial number as attribute
2. noun + (can or may) have + nouns -> first is entity, others are attributes. If there is a quantity, add the two in the cardinality array
3. db noun + (can or may) have + nouns -> the first is not considered since associated with db (e.g. 'database', 'system', etc.),others can be entity types

In [ ]:
# common nouns associated to db
db_nouns = ['database', 'db', 'system', 'organization', 'application', 'information']

def attr_ent_extraction(sentences, ent_attr, cardinality):
  # do not execute if no nouns have been found
  if len(nouns) > 0:
    for idx, sentence in enumerate(sentences):
      doc = nlp(sentence)
      i = 0
      increment = 0
      rule1 = False
      rule2 = False
      rule3 = False
      while i < len(doc):
        elem = ''
        # 3. HAVE CASE WITH DB
        if str(doc[i]) in db_nouns:
          rule3 = True
        # find current token in first token of all nouns extracted (check string equality and position)
        for elems in lemmatized_nouns[idx]:
          # find current token in nouns using position in sentence, thus no need to compare tokens
          if i == elems[1]:
            # if found, move i to the position of the last token (since it is the one having as .dep_ 'poss' and next token ''s' will be 'case') -> pick length stored of noun from nouns array
            noun_len = len(elems[0].split(' '))
            i += noun_len - 1
            # 1. POSSESSIVE CASE -> check if last token has .dep_ 'poss' and next token is 'case' = 's (and if next token exists)
            if doc[i].dep_ == 'poss' and i+1 < len(doc) and doc[i+1].dep_ == 'case':
              elem = elems[0]
              increment = 2
              rule1 = True
              # !! insert found noun in attribute dict as entity if not present
              if elems[0] not in ent_attr.keys():
                ent_attr[elem] = []
              break
            # 2. (CAN or MAY) HAVE CASE
            elif i+1 < len(doc):
              # put token to infinite
              lemm_token = doc[i+1].lemma_
              # if token = have verb, subsequent nouns are attributes
              if lemm_token == 'have' or ((lemm_token == 'can' or lemm_token == 'may') and i+2 < len(doc) and doc[i+2].lemma_ == 'have'):
                if lemm_token == 'have':
                  increment += 2
                else:
                  increment += 3
                elem = elems[0]
                if elem not in db_nouns:
                  if elem not in ent_attr.keys():
                    ent_attr[elem] = []
                rule2 = True
                break
        # noun with possible associated attributes has been found or not
        if rule1 or rule2 or rule3:
          # move i to the position of the next-next token (token after 's)
          i += increment
          # pick sequent noun word from nouns array and check if that noun is the one immediately after 's (check only first token)
          j = i
          while j < len(doc):
            # if list of attributes, jump ',' or 'and'. If rule 2, there can be determinants before nouns or a certain amount or one of the abbreviations (this condition cannot take into consideration ca.)
            if str(doc[j]) == ',' or str(doc[j]) == 'and' or ((rule2 or rule3) and (doc[j].pos_ == 'DET' or doc[j].pos_ == 'NUM' or doc[j].pos_ == 'ADJ' or str(doc[j]) in abbreviations)):
              j += 1
            # special case for ca. since it is seen as two tokens 'ca' and '.'
            elif str(doc[j]) == 'ca' and j+1 < len(doc) and str(doc[j+1]) == '.':
              j += 2
            elif doc[j].pos_ == 'NOUN' or doc[j].pos_ == 'PROPN' or str(doc[j]) == 'last' or str(doc[j]) == 'first' or str(doc[j]) == 'middle' or str(doc[j]) == 'serial':
              attr = ''
              # check if noun in extracted nouns in specific sentence to get it as attribute -> compare only position in sentence s.t. it is more performant
              for elems in lemmatized_nouns[idx]:
                if j == elems[1]:
                  attr = elems[0]
                  break
              # insert noun in attribute list and move j forward depending on the length of the noun to get the next attributes if any
              if attr != '':
                # if rule3, attributes in reality are the entities associated with the db
                if (rule3):
                  if attr not in ent_attr.keys():
                    ent_attr[attr] = []
                else:
                  if attr not in ent_attr[elem]:
                    ent_attr[elem].append(attr)
              j += len(attr.split(' '))

            # if attributes finished, update position of i at position of j and close loop and continue again with i (scanning for other nouns followed by attributes)
            else:
              i = j + 1
              break
          rule1 = False
          rule2 = False
          rule3 = False
          increment = 0
        i+= 1
  return ent_attr, cardinality

Extract entities and attributes based on the heuristic rule of possessive:
1. Possessive -> noun + of + noun -> first is the attribute, second is/are the entities, e.g. 'the name of the student', 'the names of the students and the customers'. If any, require to modify the entities extracted if noun is subject of the sentence, since 'the name of the student', name is the subject but it is the attribute of student. It is required to pay attention to some cases in which the 'of' doesn't specify an entity and an attribute:
- number of -> number of students, number of pages, etc.
- level of -> level of education, level of complexity, etc.

In [ ]:
def attr_ent_extraction_possessive(sentences, ent_attr):
  # do not execute if no nouns have been found
  if len(nouns) > 0:
    for idx, sentence in enumerate(sentences):
      doc = nlp(sentence)
      i = 0
      subject = ''
      attribute = ''
      rule1 = False
      while i < len(doc):
        elem = ''
        # find current token in first token of all nouns extracted (check string equality and position)
        for elems in lemmatized_nouns[idx]:
          # find current token in nouns using position in sentence, thus no need to compare tokens
          if i == elems[1]:
            # if found, move i to the position of the last token (since it is the one having as .dep_ 'poss' and next token ''s' will be 'case') -> pick length stored of noun from nouns array
            noun_len = len(elems[0].split(' '))
            i += noun_len - 1
            elem = elems[0]
            # RULE 1. POSSESSIVE WITH 'OF'
            if i+1 < len(doc) and str(doc[i+1]).lower() == 'of':
              rule1 = True
              attribute = elem
              # remove noun before 'of' from entities (ent_attr dict) and save it as attribute of the next noun/s
              if elem in ent_attr.keys():
                del ent_attr[elem]
              break

        # if attribute associated with entity found
        if rule1:
          # move i to the position of the next-next token (token after of)
          i += 2
          # pick sequent noun word from nouns array and check if that noun is the one immediately after 's (check only first token)
          j = i
          found = False
          noun_found = 0
          while j < len(doc):
            if noun_found == 2:
              noun_found = 0
              break
            if found:
              found = False
              break
            # if list of attributes, jump ',' or 'and' or determinants.
            if doc[j].pos_ == 'DET' or doc[j].pos_ == 'CCONJ' or str(doc[j]) == ',' or str(doc[j]).lower() == 'of':
              j += 1
            elif doc[j].pos_ == 'NOUN' or doc[j].pos_ == 'PROPN' or str(doc[j]) == 'last' or str(doc[j]) == 'first' or str(doc[j]) == 'middle' or str(doc[j]) == 'serial':
              # check if noun in extracted nouns in specific sentence to get it as attribute -> compare only position in sentence s.t. it is more performant
              for elems in lemmatized_nouns[idx]:
                if j == elems[1]:
                  entity = elems[0]
                  noun_found = 1
                  j += len(entity.split(' '))
                  # attributes and not entity-attribute relationship (number of copies, level of complexity, type of people, kind of people, piece of masterpiece, amount of money)
                  if attribute == 'number' or attribute == 'level' or attribute == 'type' or attribute == 'kind' or attribute == 'piece' or attribute == 'amount' or attribute == 'degree' or attribute == 'quality' or attribute == 'variety' or attribute == 'state':
                    attr = attribute + 'of' + entity
                  # nouns which are not attributes or nouns -> example of problems, sort of problem, etc.
                  elif attribute == 'example' or attribute == 'sort':
                    found = True
                    noun_found = 0
                    break
                  else:
                    # update attributes dict with the new entity and the associated attribute found previously
                    if entity not in ent_attr.keys():
                      ent_attr[entity] = [attribute]
                    else:
                      ent_attr[entity].append(attribute)
                  found = True
                  noun_found = 0
                  break
              if noun_found == 0:
                noun_found = 2
                break

            # if attributes finished, update position of i at position of j and close loop and continue again with i (scanning for other nouns followed by attributes)
            else:
              i = j
              break
          rule1 = False
        i+= 1
  return ent_attr

Extract entities and attributes based on one heuristic rules:

Multiple attributes appearing in a sentence will be linked to the first entity that appears in that sentence (applicable if only one entity is present in that sentence). Generally, the first entity is the subject.

Find subject, and if there are subsequent nouns, then they're attributes associated to that subject entity.

In [ ]:
# -------------------- ER ATTRIBUTES + ENTITIES EXTRACTION 3 --------------------
def attr_ent_extraction3(sentences, ent_attr, cardinality_attr, composite_attr):
  # do not execute if no nouns have been found
  if len(subjects) > 0:
    for i in range(len(sentences)):
      doc = nlp(sentences[i])
      j = 0
      # iterate over subjects
      while j < len(subjects[i]):
        is_attr = False
        found = False
        # for each subject, analyse tokens after the subject
        k = subjects[i][j][2] + 1
        for key, values in ent_attr.items():
          for value in values:
            if value == subjects[i][j][0]:
              found = True
              break
        # check if subject is an attributem, thus subsequent nouns are simple attribute of this composite attribute
        if subjects[i][j][0] in common_attr or found:
          is_attr = True
        positions = [x[2] for x in subjects[i]]
        while k < len(doc):
          # get positions of all subjects in the current sentence. If current k correspond to position of one entity, skip everything and start with new subject
          if k in positions:
            break
          noun = ''
          for elems in nouns[i]:
            if k == elems[1]:
              noun = elems[0]
              k += len(noun.split(' ')) - 1
          if noun != '':
            # lemmatize noun
            nn = nlp(noun)
            lem_token = ' '.join([token.lemma_ for token in nn])

            # if subject is in db nouns, then other nouns are entities, otherwise other nouns are attributes
            if subjects[i][j][0] in db_nouns:
              if subjects[i][j][0] not in ent_attr.keys():
                ent_attr[subjects[i][j][0]] = []
            # if subject is attribute, then it is a composite attribute formed by the subsequent nouns -> name is composed by first name, second name.
            else:
              # check for composite nouns
              if is_attr:
                if subjects[i][j][0] not in composite_attr.keys():
                  composite_attr[subjects[i][j][0]] = [lem_token]
                else:
                  composite_attr[subjects[i][j][0]].append(lem_token)
              elif subjects[i][j][0] not in ent_attr.keys():
                ent_attr[subjects[i][j][0]] = [lem_token]
              else:
                if lem_token not in ent_attr[subjects[i][j][0]]:
                  ent_attr[subjects[i][j][0]].append(lem_token)


              # if attribute is plural, then it is multivalued and insert in cardinality_attr. For composite nouns check last token
                last_tok = nn[-1]
                if len(last_tok.morph.get('Number')) > 0 and last_tok.morph.get('Number')[0] == 'Plur':
                  elem = [2, subjects[i][j][0], lem_token]
                  if elem not in cardinality_attr:
                    cardinality_attr.append(elem)
            noun = ''
          k += 1
        j += 1

  return ent_attr, cardinality_attr, composite_attr

Unique attribute extraction

If an entity doesn't have a unique attribute and contains attributes which usually are considered unique (id, name, ssn), select them as unique attributes

In [ ]:
common_unique_attr = ['id', 'identification number', 'ssn', 'code', 'name']

In [ ]:
def add_unique(ent_attr, cardinality_attr):
  # check if entity not in cardinality attr
  for key, values in ent_attr.items():
    found = False
    for elem in cardinality_attr:
      if key in elem and elem[0] == 1:
        found = True
        break
    # if not found, check in its attributes if one is in the list of common unique attributes
    if not found:
      attributes = []
      for attr in common_unique_attr:
        if attr in values or attr.lower() in values:
          attributes.append(attr)
      # if there is an attribute/s in the list of attributes which can be used as unique, select it and add in cardinality_attr
      # since only one can be selected, select them in order: id, ssn, code, name
      if len(attributes) != 0:
        if 'id' in attributes or 'ID' in attributes:
          cardinality_attr.append([1, key, 'ID'])
        elif 'identification number' in attributes:
          cardinality_attr.append([1, key, 'identification number'])
        elif 'ssn' in attributes:
          cardinality_attr.append([1, key, 'ssn'])
        elif 'code' in attributes:
          cardinality_attr.append([1, key, 'code'])
        elif 'name' in attributes:
          cardinality_attr.append([1, key, 'name'])
  return cardinality_attr

## Extraction methods cleaning

Remove all nouns associated with db both from entities and attributes

In [ ]:
def db_nouns_clean(dictionary):
  dic_copy = dictionary.copy()
  for key, value in dic_copy.items():
    if key in db_nouns:
      del dictionary[key]
    else:
      for elem in value:
        if elem in db_nouns:
          dictionary[key].remove(elem)

Remove nouns from attributes list if in list of attributes but also key (entity) in ent_attr

In [ ]:
def ent_attr_clean(dictionary):
  dict_new = {}
  for key, values in dictionary.items():
    list_attr = values.copy()
    for i in range(0, len(values)):
      if values[i] in dictionary.keys():
        list_attr.remove(values[i])
    dict_new[key] = list_attr
  # if word combined by name of entity and attribute and also present uniquely as entity, remove it as attribute
  # e.g. {'car': ['owner'], 'car owner': []}, remove 'owner' from 'car'
  dict_new2 = {}
  for key, values in dict_new.items():
    list_attr = values
    for value in values:
      composite_nn = key + ' ' + value
      if composite_nn in dict_new.keys():
        list_attr.remove(value)
    dict_new2[key] = list_attr
  return dict_new2

Remove from dictionary of composite attributes nouns which are not composite attributes but are in the list of entities

In [ ]:
def composite_attr_clean(dictionary, ent_attr):
  dict_new = {}
  for key, values in dictionary.items():
    if key not in ent_attr.keys():
      dict_new[key] = values
  return dict_new

Remove from the array of cardinalities for the attributes the elements in which both are entities.

e.g. [[2, 'project', 'professor'], [2, 'professor', 'phone number']] -> [2, 'professor', 'phone number']]

In [ ]:
def card_attr_clean(attrs, ent_attr):
  new_attrs = []
  for elem in attrs:
    # first noun is always an entity, check if second one is an entity. If yes, remove
    if elem[2] not in ent_attr.keys():
      new_attrs.append(elem)
  return new_attrs

# Methods combinations

Different combinations of methods will be used, and based on three parameters (precision, recall, F1 or accuracy) the best combination will be selected.

The combination will be tested on a training set, and when the fine-tuning is completed, the best one will be tested on a test set (only once).

## Performance measures:
Computed only with respect to entities and attributes

Precision: true positives/ (true positives + false positives)



In [ ]:
# the predict_set_cardinality contains the cardinality, the two entities and the verb of the relationship.
# For evaluation we remove the verb and the cardinality maintaing only the entities of the relationship

# make copy of cardinality to modify it while checking (there can be more than one relationships between two entities and these copies should be taken into account, thus whenever one is found remove it from the list)

# ent_attr is a dictionary containing entities as keys and attributes as values
def precision(true_ent_attr, predict_ent_attr, true_set_cardinality = None, predict_set_cardinality = None, true_unique_attr = None, predict_unique_attr = None):
  true_positives = 0
  false_positives = 0


  # check if key is present, if yes check for each value, otherwise false_positives increased by the key and all the values (since if the key is not present its attributes are not present as well)
  if len(predict_ent_attr) > 0:
    for key, values in predict_ent_attr.items():
      if key in true_ent_attr.keys():
        true_positives += 1
        for value in values:
          if value in true_ent_attr[key]:
            true_positives += 1
          else:
            false_positives += 1
      else:
        qnt = len(predict_ent_attr[key])
        false_positives += qnt + 1

  if true_set_cardinality != None and len(true_set_cardinality) > 0:
    card_true_copy = true_set_cardinality.copy()

    for cardinality in predict_set_cardinality:
      sub_cardinality = [cardinality[1], cardinality[2]]
      if sub_cardinality in card_true_copy:
        true_positives += 1
        card_true_copy.remove(sub_cardinality)
      else:
        false_positives += 1

  if true_unique_attr != None and len(true_unique_attr) > 0:
    for elem in predict_unique_attr:
      if elem in true_unique_attr:
        true_positives += 1
      else:
        false_positives += 1

  if true_positives + false_positives == 0:
    return 0
  else:
    return true_positives / (true_positives + false_positives)

Recall: true positives / (true positives + false negatives)

In [ ]:
def recall(true_ent_attr, predict_ent_attr, true_set_cardinality = None, predict_set_cardinality = None, true_unique_attr = None, predict_unique_attr = None):
  true_positives = 0
  false_negatives = 0

  sub_predict_set_cardinality = []
  card_true_copy = None
  card_predict_copy = None

  if true_set_cardinality != None and len(true_set_cardinality) > 0:
    card_true_copy = true_set_cardinality.copy()
    for cardinality in predict_set_cardinality:
      sub_cardinality = [cardinality[1], cardinality[2]]
      sub_predict_set_cardinality.append(sub_cardinality)

    card_predict_copy = sub_predict_set_cardinality.copy()


  if predict_ent_attr != None and len(predict_ent_attr) > 0:
    for key, values in predict_ent_attr.items():
      if key in true_ent_attr.keys():
        true_positives += 1
        for value in values:
          if value in true_ent_attr[key]:
            true_positives += 1

    for key, values in true_ent_attr.items():
      if key not in predict_ent_attr.keys():
        qnt = len(true_ent_attr[key])
        false_negatives += qnt + 1
      else:
        for value in values:
          if value not in predict_ent_attr[key]:
            false_negatives += 1

  if true_set_cardinality != None and len(true_set_cardinality) > 0:
    for cardinality in predict_set_cardinality:
      sub_cardinality = [cardinality[1], cardinality[2]]
      if sub_cardinality in card_true_copy:
        true_positives += 1
        card_true_copy.remove(sub_cardinality)

    for true_cardinality in true_set_cardinality:
      if true_cardinality not in card_predict_copy:
        false_negatives += 1
      else:
        if true_cardinality in card_predict_copy:
          card_predict_copy.remove(true_cardinality)

  if true_unique_attr != None and len(true_unique_attr) > 0:
    for elem in predict_unique_attr:
      if elem in true_unique_attr:
        true_positives += 1

    for elem in true_unique_attr:
      if elem not in predict_unique_attr:
        false_negatives += 1

  if true_positives + false_negatives == 0:
    return 0
  else:
    return true_positives / (true_positives + false_negatives)

F1: 2* ((precision * recall)/ (precision + recall))

In [ ]:
def f1(precision, recall):
  if precision + recall == 0:
    return 0
  else:
    return 2 * ((precision * recall) / (precision + recall))

## Combinations
The pre-processing part is the same for all combinations.
The combinations consider only few and secondary (less precise) methods, since the main ones can be used in order paying attention to conflicts.

Combinations:
1. Initial combination (without adding the few less precise methods)
2. Initial combination + underscore
3. Initial combination + underscore + frequency
4. Initial combination + underscore + frequency + subjects

For each of these combinations, there are two results for the measurements:
1. an additional cleaning not-included
2. an additional cleaning included

The additional cleaning refers to the function ent_attr_clean() for the ent_attr dictionary.

### Pre-processing (same for all combinations)

In [ ]:
input_text = text
# simplify english text through GPT
input_text = gpt_output()
print(input_text)
# clean unnecessary spaces, backticks, etc.
input_text = clean_gpt_output(input_text)
print(input_text)
# sentence segmentation
input_text = sentence_segmentation_mine(input_text)
# transform into lowercase
input_text = to_lowercase(input_text)
# remove pronouns if any
subject_extraction_for_pron(input_text)
no_pronoun_text = pronoun_substitution(input_text)

# lemmatize text (not always used in the functions)
lemmatizedText = text_lemmatization(input_text)

'

A company operates four departments. Each department employs one or more employees. An employee may have one or more dependents. An employee may have an employment history.

'
A company operates four departments. Each department employs one or more employees. An employee may have one or more dependents. An employee may have an employment history.
['A company operates four departments.', 'Each department employs one or more employees.', 'An employee may have one or more dependents.', 'An employee may have an employment history.']


### Starting combination
Same initial methods for all combinations (the main ones can be applied in a specific order)

In [ ]:
# list of non-lemmatized nouns
nouns = nouns_extraction(input_text)
# remove proper nouns of locations and people
nouns = proper_noun_removal(nouns)

# list of lemmatized nouns
lemmatized_nouns = noun_lemmatization(nouns)

# get lemmatized subjects of entire text
subjects = subj_extraction(lemmatized_nouns)

In [ ]:
def starting_comb():
  # dictionary having entities as keys and list of attributes as values
  # e.g. {'person': ['id', 'sex'], 'student': ['number']}
  ent_attr = {}

  # array having as element [cardinality, entity, attribute], for entity/attribute relationships (unique or multi-valued attributes),
  # e.g. [1,person,id] -> a person has a unique id
  # e.g. [2, person, phone number] -> a person has multiple phone numbers
  cardinality_attr = []

  # array having as elements [cardinality, first entity, second entity, verb of relationship] for entity/entity relationships.
  # e.g. [('1', '2'), 'user', 'website', ' uses'] -> a user uses multiple websites
  cardinality = []

  # dictionary containing as key the entity and as values the first as the composite attribute and the others the simple attribute composing the composite one
  # e.g. {'person': ['name', 'first name', 'last name']}
  composite_attr = {}

  # 1. basic combination of methods

  # insert in ent_attr
  # extract from composite nouns common attributes, e.g. 'person id'
  ent_attr = extract_common(lemmatized_nouns, ent_attr)

  # insert in cardinality_attr
  # extract attributes in sentences 'attribute's entity is unique.', e.g. 'person's id is unique'
  cardinality_attr = unique_extr(no_pronoun_text, cardinality_attr)

  # insert in ent_attr
  # extract entities and attributes if possessive form 'of', e.g. 'the name of the person'
  ent_attr = attr_ent_extraction_possessive(lemmatizedText, ent_attr)

  # insert in ent_attr, cardinality_attr, cardinality
  # extract entities and their relationship based on quantity tokens, e.g. 'a writer has written books'
  ent_attr, cardinality_attr, cardinality = quantity_extr(no_pronoun_text, subjects, ent_attr, cardinality_attr, cardinality)

  # insert in ent_attr, cardinality
  # extract entities based on possessive or 'have' verb
  ent_attr, cardinality = attr_ent_extraction(lemmatizedText, ent_attr, cardinality)

  # insert in ent_attr, cardinality_attr
  # extract entities and attributes based on the fact that if a subject (entity) is followed by some nouns, these ones are its attributes
  # result in ent_attr in conflict with ent_attr4, thus remove here the attribute which are seen as entities in the cardinality1
  ent_attr, cardinality_attr, composite_attr = attr_ent_extraction3(no_pronoun_text, ent_attr, cardinality_attr,composite_attr)

  # clean dictionary of composite attributes by removing the entities
  composite_attr = composite_attr_clean(composite_attr, ent_attr)

  return ent_attr, cardinality, cardinality_attr, composite_attr

Evaluation basic combination without and with additional cleaning

In [ ]:
str_ent_attr, str_cardinality, str_cardinality_attr, str_composite_attr = starting_comb()
# remove nouns associated to db
db_nouns_clean(str_ent_attr)

prec = precision(true_res, str_ent_attr)
rec = recall(true_res, str_ent_attr)
f_score1 = f1(prec, rec)

print('true dict:', true_res, '\npredicted dict:', str_ent_attr)
print('basic precision:', prec)
print('basic recall:', rec)
print('basic F1:', f_score1)

In [ ]:
str_ent_attr = ent_attr_clean(str_ent_attr)

prec = precision(true_res, str_ent_attr)
rec = recall(true_res, str_ent_attr)
f_score1 = f1(prec, rec)

print('true dict:', true_res, '\npredicted dict:', str_ent_attr)
print('basic precision with additional cleaning:', prec)
print('basic recall with additional cleaning:', rec)
print('basic F1 with additional cleaning:', f_score1)

### Combination 1
basic combo +
1. extract_underscore

In [ ]:
ent_attr1, cardinality1, cardinality_attr1, composite_attr1 = starting_comb()

ent_attr1 = extract_underscore(nouns, ent_attr1)
db_nouns_clean(ent_attr1)

prec1 = precision(true_res, ent_attr1)
rec1 = recall(true_res, ent_attr1)
f11 = f1(prec1, rec1)

print('true dict:', true_res, '\n predicted dict:', ent_attr1)
print('basic precision:', prec1)
print('basic recall:', rec1)
print('basic F1:', f11)

In [ ]:
ent_attr1 = ent_attr_clean(ent_attr1)

prec1 = precision(true_res, ent_attr1)
rec1 = recall(true_res, ent_attr1)
f11 = f1(prec1, rec1)

print('true dict:', true_res, '\n predicted dict:', ent_attr1)
print('basic precision with additional cleaning:', prec1)
print('basic recall with additional cleaning:', rec1)
print('basic F1 with additional cleaning:', f11)

### Combination 2
basic combo +
1. extract_underscore
2. freq_extraction -> in general entities have highest frequency, thus remove entities not enough frequent

In [ ]:
ent_attr2, cardinality2, cardinality_attr2, composite_attr2 = starting_comb()
ent_attr2 = extract_underscore(nouns, ent_attr2)
print(ent_attr2)
freq_entities2, freq_attibutes2 = freq_extraction(nouns)

Remove entities if not in frequency array (the entity doesn't have a high enough frequency to be considered as entity)

In [ ]:
def filter_entities(dictionary, frequency):
  dictionary_copy = dictionary.copy()
  for key in dictionary_copy.keys():
    if key not in frequency:
      del dictionary[key]
  return dictionary

In [ ]:
ent_attr2 = filter_entities(ent_attr2, freq_entities2)
db_nouns_clean(ent_attr2)

In [ ]:
prec2 = precision(true_res, ent_attr2)
rec2 = recall(true_res, ent_attr2)
f12 = f1(prec2, rec2)

print('true dict:', true_res, '\n predicted dict:', ent_attr2)
print('basic precision:', prec2)
print('basic recall:', rec2)
print('basic F1:', f12)

In [ ]:
ent_attr_final2 = ent_attr_clean(ent_attr2)

prec2 = precision(true_res, ent_attr2)
rec2 = recall(true_res, ent_attr2)
f12 = f1(prec2, rec2)

print('true dict:', true_res, '\n predicted dict:', ent_attr2)
print('basic precision with additional cleaning:', prec2)
print('basic recall with additional cleaning:', rec2)
print('basic F1 with additional cleaning:', f12)

### Combination 3
basic combo +
1. extract_underscore
2. freq_extraction
3. subj_extraction -> in general entities have been also at least once the subject of a sentence

In [ ]:
ent_attr3, cardinality3, cardinality_attr3, composite_attr3 = starting_comb()
ent_attr3 = extract_underscore(nouns, ent_attr3)

freq_entities3, freq_attibutes3 = freq_extraction(nouns)

ent_attr3 = filter_entities(ent_attr3, freq_entities3)

Remove entities if they have not been subjects of a sentence at least once

In [ ]:
# take from array of subjects (structure e.g. [[['user', 'NN', 3]], [['ID', 'NNP', 3]], [['user', 'NN', 1]], [['website', 'NN', 1]]]) only subjects without repetition -> ['user', 'ID', 'website']
def extract_subjects(subj_array):
  subj_only = []
  for sentence in subj_array:
    for elem in sentence:
      if elem[0] not in subj_only:
        subj_only.append(elem[0])
  return subj_only

In [ ]:
simplified_subjs = extract_subjects(subjects)

In [ ]:
ent_attr3 = filter_entities(ent_attr3, simplified_subjs)
db_nouns_clean(ent_attr3)

In [ ]:
prec3 = precision(true_res, ent_attr3)
rec3 = recall(true_res, ent_attr3)
f13 = f1(prec3, rec3)

print('true dict:', true_res, '\n predicted dict:', ent_attr3)
print('basic precision:', prec3)
print('basic recall:', rec3)
print('basic F1:', f13)

true dict: {'member': ['memdid', 'name', 'zip', 'date'], 'membership type': ['mname', 'price', 'mid'], 'sale transaction': ['date', 'tid'], 'merchandise item': ['mrchid', 'price', 'name'], 'day pass': ['ID', 'date'], 'pass category': ['passcatid', 'pcname', 'price']} 
 predicted dict: {'employee': ['employment history', 'dependent']}
basic precision: 0.0
basic recall: 0.0
basic F1: 0


In [ ]:
ent_attr3 = ent_attr_clean(ent_attr3)

prec3 = precision(true_res, ent_attr3)
rec3 = recall(true_res, ent_attr3)
f13 = f1(prec3, rec3)

print('predicted dict:', ent_attr3)
print('basic precision with additional cleaning:', prec3)
print('basic recall with additional cleaning:', rec3)
print('basic F1 with additional cleaning:', f13)

predicted dict: {'employee': ['employment history', 'dependent']}
basic precision with additional cleaning: 0.0
basic recall with additional cleaning: 0.0
basic F1 with additional cleaning: 0


### Combination 4
basic combo +
1. extract_underscore
2. remove entities only if they're not enough frequent (freq_extraction) and they have never been subjects (subj_extraction)

In [ ]:
ent_attr4, cardinality4, cardinality_attr4, composite_attr4 = starting_comb()
ent_attr4 = extract_underscore(nouns, ent_attr4)

freq_entities4, freq_attibutes4 = freq_extraction(nouns)

simplified_subjs1 = extract_subjects(subjects)

get only nouns which are both in frequency array and subject array

In [ ]:
def merge(arr1, arr2):
  merged = []
  for elem in arr1:
    if elem in arr2:
      merged.append(elem)
      break

In [ ]:
common_nouns = merge(freq_entities4, simplified_subjs1)

remove only entities which are not in both frequency and subject

In [ ]:
if common_nouns != None:
  ent_attr4 = filter_entities(ent_attr4, common_nouns)
db_nouns_clean(ent_attr4)

In [ ]:
prec4 = precision(true_res, ent_attr4)
rec4 = recall(true_res, ent_attr4)
f14 = f1(prec4, rec4)

print('true dict:', true_res, '\n predicted dict:', ent_attr4)
print('basic precision:', prec4)
print('basic recall:', rec4)
print('basic F1:', f14)

In [ ]:
ent_attr_final4 = ent_attr_clean(ent_attr4)

prec4 = precision(true_res, ent_attr4)
rec4 = recall(true_res, ent_attr4)
f14 = f1(prec4, rec4)

print('true dict:', true_res, '\n predicted dict:', ent_attr4)
print('basic precision with additional cleaning:', prec4)
print('basic recall with additional cleaning:', rec4)
print('basic F1 with additional cleaning:', f14)

# Evaluation combinations using training set

Results (precision, recall and F1 score for each sample in training set)

In [ ]:
basic = [([0.5, 0.487, 0.493], [0.594, 0.487, 0.535]), ([0.75, 1.0, 0.857], [0.875, 1.0, 0.933]), ([0.533, 0.667, 0.593], [0.571, 0.667, 0.615]), ([0.413, 0.95, 0.576], [0.542, 0.95, 0.691]),([0.8, 1.0, 0.889], [1.0, 1.0, 1.0]),([0.375, 0.75, 0.5], [0.426, 0.75, 0.545]),([0.714, 0.833, 0.769], [0.833, 0.833, 0.833]),([0.474, 0.75, 0.581], [0.533, 0.667, 0.592]),([0.3, 1.0, 0.461], [0.5, 1.0, 0.667]),([0.714, 0.909, 0.8], [0.833, 0.909, 0.869]),([0.778, 1.0, 0.875], [0.875, 1.0, 0.933]),([0.75, 0.947, 0.837], [0.857, 0.947, 0.9]),([0.538, 0.636, 0.583], [0.7, 0.636, 0.667]),([0.7, 0.913, 0.792], [0.84, 0.913, 0.875]),([0.667, 1.0, 0.8], [0.8, 1.0, 0.889]),([0.857, 1.0, 0.923], [1.0, 1.0, 1.0]),([0.55, 0.846, 0.667], [0.611, 0.846, 0.71]),([0.5, 0.75, 0.6], [0.75, 0.75, 0.75]),([0.64, 0.727, 0.682], [0.75, 0.682, 0.714])]
one = [([0.5, 0.487, 0.493], [0.594, 0.487, 0.535]), ([0.75, 1.0, 0.857], [0.875, 1.0, 0.933]), ([0.421, 0.667, 0.516], [0.444, 0.667, 0.533]),([0.413, 0.95, 0.576], [0.542, 0.95, 0.691]),([0.8, 1.0, 0.889], [1.0, 1.0, 1.0]),([0.375, 0.75, 0.5], [0.426, 0.75, 0.545]),([0.714, 0.833, 0.769], [0.833, 0.833, 0.833]),([0.391, 0.75, 0.514], [0.47, 0.667, 0.552]),([0.3, 1.0, 0.461], [0.5, 1.0, 0.667]),([0.714, 0.909, 0.8], [0.833, 0.909, 0.869]),([0.778, 1.0, 0.875], [0.875, 1.0, 0.933]),([0.75, 0.947, 0.837], [0.857, 0.947, 0.9]),([0.538, 0.636, 0.583], [0.7, 0.636, 0.667]),([0.7, 0.913, 0.792], [0.84, 0.913, 0.875]),([0.667, 1.0, 0.8], [0.8, 1.0, 0.889]),([0.6, 1.0, 0.75], [0.67, 1.0, 0.8]),([0.55, 0.846, 0.667], [0.611, 0.846, 0.71]),([0.5, 0.75, 0.6], [0.75, 0.75, 0.75]),([0.64, 0.727, 0.681], [0.75, 0.682, 0.714])]
two = [([0.5, 0.487, 0.493 ], [0.5, 0.487, 0.493]),([0.808, 1.0, 0.894], [0.808, 1.0, 0.894]), ([0.533, 0.667, 0.593], [0.533, 0.667, 0.593]),([0.421, 0.8, 0.552], [0.421, 0.8, 0.552]),([0.8, 1.0, 0.889], [0.8, 1.0, 0.889]),([0.333, 0.5, 0.4], [0.333, 0.5, 0.4]),([0.714, 0.833, 0.769], [0.714, 0.833, 0.769]),([0.529, 0.75, 0.62], [0.529, 0.75, 0.62]),([0.3, 1.0, 0.461], [0.3, 1.0, 0.461]),([0.714, 0.909, 0.8], [0.714, 0.909, 0.8]),([0.778, 1.0, 0.875], [0.778, 1.0, 0.875]),([0.75, 0.947, 0.837], [0.75, 0.947, 0.837]),([0.7, 0.636, 0.667], [0.7, 0.636, 0.667]),([0.75, 0.913, 0.823], [0.75, 0.913, 0.823]),([1.0, 1.0, 1.0], [1.0, 1.0, 1.0]),([0.857, 1.0, 0.923], [0.857, 1.0, 0.923]),([0.75, 0.692, 0.72], [0.75, 0.692, 0.72]),([0.5, 0.75, 0.6], [0.5, 0.75, 0.6]),([0.64, 0.727, 0.681], [0.64, 0.727, 0.681])]
three = [([0.5, 0.487, 0.493], [0.594, 0.487, 0.535]),([0.808, 1.0, 0.894], [0.9545, 1.0, 0.977]), ([0.533, 0.667, 0.593], [0.571, 0.667, 0.615]),([0.421, 0.8, 0.552], [0.526, 0.8, 0.627]),([0.8, 1.0, 0.889], [1.0, 1.0, 1.0]),([0.333, 0.5, 0.4], [0.333, 0.5, 0.4]),([0.714, 0.833, 0.769], [0.833, 0.833, 0.833]),([0.529, 0.75, 0.62], [0.6, 0.75, 0.667]),([0.3, 1.0, 0.461], [0.5, 1.0, 0.667]),([0.714, 0.909, 0.8], [0.833, 0.909, 0.869]),([0.778, 1.0, 0.875], [0.875, 1.0, 0.933]),([0.75, 0.947, 0.837], [0.857, 0.947, 0.9]),([0.7, 0.636, 0.667], [1.0, 0.636, 0.778]),([0.75, 0.913, 0.823], [0.913, 0.913, 0.913]),([1.0, 1.0, 1.0], [1.0, 1.0, 1.0]),([0.857, 1.0, 0.923], [1.0, 1.0, 1.0]),([0.75, 0.692, 0.72], [0.818, 0.692, 0.75]),([0.5, 0.75, 0.6], [0.75, 0.75, 0.75]), ([0.64, 0.727, 0.681],[0.75, 0.682, 0.714])]
four = [([0.5, 0.487, 0.493], [0.5, 0.487, 0.493]),([0.75, 1.0, 0.857], [0.75, 1.0, 0.857]), ([0.421, 0.667, 0.516], [0.421, 0.667, 0.516]),([0.413, 0.95, 0.576], [0.413, 0.95, 0.575]),([0.8, 1.0, 0.889], [0.8, 1.0, 0.889]),([0.375, 0.75, 0.5], [0.375, 0.75, 0.5]),([0.714, 0.833, 0.769], [0.714, 0.833, 0.769]),([0.391, 0.75, 0.514], [0.391, 0.75, 0.514]),([0.3, 1.0, 0.461], [0.3, 1.0, 0.461]),([0.714, 0.909, 0.8], [0.714, 0.909, 0.8]),([0.778, 1.0, 0.875], [0.778, 1.0, 0.875]),([0.75, 0.947, 0.837], [0.75, 0.947, 0.837]),([0.538, 0.636, 0.583], [0.538, 0.636, 0.583]),([0.7, 0.913, 0.792], [0.7, 0.913, 0.792]),([0.667, 1.0, 0.8], [0.667, 1.0, 0.8]),([0.6, 1.0, 0.75], [0.6, 1.0, 0.75]),([0.55, 0.846, 0.667], [0.55, 0.846, 0.667]),([0.5, 0.75, 0.6], [0.5, 0.75, 0.6]),([0.64, 0.727, 0.681], [0.64, 0.727, 0.681])]

Average precision, recall and F1 score of all samples in training set

In [ ]:
def avg(combination):
  avg_res = []
  standard = [0,0,0]
  add_clean = [0,0,0]
  tot_nr = len(combination)
  for elem in combination:
    standard[0] += elem[0][0]
    standard[1] +=  elem[0][1]
    standard[2] += elem[0][2]

    add_clean[0] += elem[1][0]
    add_clean[1] += elem[1][1]
    add_clean[2] += elem[1][2]
  standard = [x/tot_nr for x in standard]
  add_clean = [x/tot_nr for x in add_clean]
  avg_res.append(standard)
  avg_res.append(add_clean)
  return avg_res

In [ ]:
avg_basic = avg(basic)
print('average precision, recall and F1 of basic:',avg_basic)

avg_one = avg(one)
print('average precision, recall and F1 of one:',avg_one)

avg_two = avg(two)
print('average precision, recall and F1 of two:',avg_two)

avg_three = avg(three)
print('average precision, recall and F1 of three:',avg_three)

avg_four = avg(four)
print('average precision, recall and F1 of four:',avg_four)

avg_basic: [[0.6080526315789473, 0.8507894736842104, 0.698842105263158], [0.7310526315789474, 0.8440526315789473, 0.7746315789473683]]
avg_one: [[0.5842631578947369, 0.8507894736842104, 0.6821052631578948], [0.7036842105263158, 0.8440526315789473, 0.7576842105263157]]
avg_two: [[0.6514210526315789, 0.8216315789473684, 0.7156315789473685], [0.6514210526315789, 0.8216315789473684, 0.7156315789473685]]
avg_three: [[0.6514210526315789, 0.8216315789473684, 0.7156315789473685], [0.774078947368421, 0.8192631578947368, 0.7856842105263159]]
avg_four: [[0.5842631578947369, 0.8507894736842104, 0.6821052631578948], [0.5842631578947369, 0.8507894736842104, 0.6820526315789474]]


We take into consideration the F1 score, since the recall is always higher than the precision, thus they're not balanced. By the results above, the best combination is the third combination with the additional cleaning (0.7856842105263159 ~= 79%)

# Best combination evaluation
Evaluation of the best combination taking into consideration attributes, entities, relationships among entities, unique/multivalued/composite attributes (keys)

In [ ]:
best_ent_attr = ent_attr3
best_cardinality = cardinality3
# maintain unique and multivalued attributes
best_cardinality_attr = cardinality_attr3
# clean cardianlity_attr
best_cardinality_attr = card_attr_clean(best_cardinality_attr, best_ent_attr)

best_composite_attr = composite_attr3

employee ['employment history', 'dependent']


unique:
 [[2, 'employee', 'dependent']]
relation:
 [[('1', '12'), 'department', 'employee', 'employs'], [('1', '12'), 'employee', 'dependent', 'have']]
composite:
 {'department': ['employee']}


[[8, 5, 3],
 [9, 6, 1],
 [3, 2, 1],
 [6, 0, 0],
 [3, 0, 0],
 [0, 1, 3],
 [0, 1, 4],
 [4, 0, 0],
 [2, 0, 0],
 [1, 2, 1]]

In [ ]:
best_precision = precision(true_res, best_ent_attr, true_card, best_cardinality, true_unique, best_cardinality_attr)
best_recall = recall(true_res, best_ent_attr, true_card, best_cardinality, true_unique, best_cardinality_attr)
best_f1 = f1(best_precision, best_recall)

print('best precision:', best_precision)
print('best recall:', best_recall)
print('best f1 score:', best_f1)

best precision: 0.0
best recall: 0.0
best f1 score: 0
[0.0, 0.0, 0]


1. Results of best combination of all samples using training set

In [ ]:
best_res = [[0.8888888888888888, 0.7441860465116279, 0.8101265822784809], [0.47058823529411764, 0.4444444444444444, 0.45714285714285713], [0.5306122448979592, 0.325, 0.40310077519379844], [0.8888888888888888, 0.7441860465116279, 0.8101265822784809], [1.0, 0.8857142857142857, 0.9393939393939393], [0.6666666666666666, 0.5, 0.5714285714285715], [0.875, 0.875, 0.875], [0.7142857142857143, 0.7692307692307693, 0.7407407407407408], [0.7333333333333333, 0.8461538461538461, 0.7857142857142856], [0.5833333333333334, 0.7, 0.6363636363636365], [0.4791666666666667, 0.7419354838709677, 0.5822784810126582], [0.7894736842105263, 0.7894736842105263, 0.7894736842105263], [0.9, 0.9310344827586207, 0.9152542372881356], [0.6875, 0.6111111111111112, 0.6470588235294118], [0.631578947368421, 0.6153846153846154, 0.6233766233766234], [0.9166666666666666, 1.0, 0.9565217391304348], [0.6, 0.8571428571428571, 0.7058823529411764], [0.7419354838709677, 0.5897435897435898, 0.6571428571428573], [0.6666666666666666, 0.5454545454545454, 0.6], [0.9, 0.9, 0.9], [0.9, 0.9, 0.9]]

Average result

In [ ]:
best_res_np = np.array(best_res)
mean_results = np.mean(best_res_np, axis=0)
mean_results[2] = f1(mean_results[0], mean_results[1])
print('precision, recall and F1 score:',mean_results)

[0.74117073 0.72929504 0.73518493]


2. Results of best combination of all samples - using test set

In [ ]:
best_res = [[0.5625, 0.4864864864864865, 0.5217391304347827], [0.7058823529411765, 0.4, 0.5106382978723405], [0.8235294117647058, 0.35, 0.4912280701754386], [0.32558139534883723, 0.2916666666666667, 0.30769230769230776], [1.0, 1.0, 1.0], [0.7857142857142857, 0.7333333333333333, 0.7586206896551724], [0.92, 0.8846153846153846, 0.9019607843137256], [0.875, 0.7777777777777778, 0.823529411764706], [0.6785714285714286, 0.6333333333333333, 0.6551724137931035], [0.6, 0.23076923076923078, 0.33333333333333337]]


Average result

In [ ]:
best_res_np = np.array(best_res)
mean_results = np.mean(best_res_np, axis=0)
mean_results[2] = f1(mean_results[0], mean_results[1])
print('precision, recall and F1 score:',mean_results)

[0.72767789 0.57879822 0.64475525]


3. result of best combination with separated evaluations - training set



In [ ]:
def precision(tp, fp):
  if tp + fp == 0:
    return 0
  else:
    return tp/(tp + fp)

In [ ]:
def recall(tp, fn):
  if tp + fn == 0:
    return 0
  else:
    return tp/(tp + fn)

In [ ]:
def f1(precision,recall):
  if precision + recall == 0:
    return 0
  else:
    return 2 * ((precision * recall) / (precision + recall))

In [ ]:
# ent
ent = [[4,0,0],[5,0,3], [6,0,0],[4,1,2],[5,2,0],[4,0,0], [3,0,1],[3,0,0],[2,0,1],[4,0,0],[3,0,0],[5,0,0],[2,1,1],[3,0,0],[3,1,0],[2,0,2],[3,0,0],[2,0,0], [1,0,0],[2,0,0],[3,0,1]]
# attr
attr = [[17,1,0],[11,9,9],[17,1,0],[11,4,3],[14,10,0],[6,1,1],[12,0,4],[10,2,2],[7,1,3],[16,1,0],[9,1,1],[15,0,0],[7,1,4],[9,0,0],[0,3,0],[0,3,0],[8,1,0],[5,1,0], [3,0,0],[4,0,0],[0,1,0]]
# unique
unique = [[3,0,1],[0,1,9],[4,0,2], [2,0,4],[4,1,1],[1,0,1],[4,0,2],[3,0,0],[3,0,0],[4,0,0],[0,0,3],[4,0,0],[0,2,0],[0,0,0],[0,0,0],[0,0,0],[3,0,0],[2,0,0],[0,0,0],[1,1,0],[0,0,0]]
# composite
composite = [[0,0,0], [0,2,0],[0,0,0],[0,1,0],[0,2,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,2,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,1,0]]
# multivalued
multivalued = [[0,1,0], [0,8,0],[0,0,0],[0,1,0],[0,5,0],[0,1,0],[1,1,0],[0,0,0],[2,0,0],[0,0,0],[0,1,0],[0,0,0],[0,0,0],[0,0,0],[0,2,0],[0,1,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]]
# relation
relation = [[7,2,2], [10,3,6],[5,0,0],[3,1,3],[4,3,1],[2,1,1],[5,1,0],[2,1,0],[0,2,3],[3,0,0],[1,2,1],[5,0,0],[1,1,3],[2,0,0],[4,0,1],[3,1,0],[2,1,1],[1,0,0],[1,2,0],[1,0,0],[3,0,0]]

In [ ]:
ent_np_prompt_train = np.array(ent)
attr_np_prompt_train = np.array(attr)
unique_np_prompt_train = np.array(unique)
composite_np_prompt_train = np.array(composite)
multivalued_np_prompt_train = np.array(multivalued)
relation_np_prompt_train = np.array(relation)

sum_ent_prompt_train = np.sum(ent_np_prompt_train, axis=0)
sum_attr_prompt_train = np.sum(attr_np_prompt_train, axis=0)
sum_unique_prompt_train = np.sum(unique_np_prompt_train, axis=0)
sum_composite_prompt_train = np.sum(composite_np_prompt_train, axis=0)
sum_multivalued_prompt_train = np.sum(multivalued_np_prompt_train, axis=0)
sum_relation_prompt_train = np.sum(relation_np_prompt_train, axis=0)

print('ent tp, fp and fn:',sum_ent_prompt_train)
print('attr tp, fp and fn:',sum_attr_prompt_train)
print('unique tp, fp and fn:',sum_unique_prompt_train)
print('composite tp, fp and fn:',sum_composite_prompt_train)
print('multivalued tp, fp and fn:',sum_multivalued_prompt_train)
print('relationships tp, fp and fn:',sum_relation_prompt_train)

ent: [69  5 11]
attr: [181  41  27]
unique: [38  5 23]
composite: [0 8 0]
multivalued: [ 3 21  0]
relationships: [65 21 22]


In [ ]:
ent_prec_train = precision(sum_ent_prompt_train[0], sum_ent_prompt_train[1])
ent_rec_train = recall(sum_ent_prompt_train[0], sum_ent_prompt_train[2])
ent_f1_train = f1(ent_prec_train, ent_rec_train)

print('ent precision, recall and F1 score:', ent_prec_train, ent_rec_train, ent_f1_train)

attr_prec_train = precision(sum_attr_prompt_train[0], sum_attr_prompt_train[1])
attr_rec_train = recall(sum_attr_prompt_train[0], sum_attr_prompt_train[2])
attr_f1_train = f1(attr_prec_train, attr_rec_train)

print('attr precision, recall and F1 score:', attr_prec_train, attr_rec_train, attr_f1_train)

unique_prec_train = precision(sum_unique_prompt_train[0], sum_unique_prompt_train[1])
unique_rec_train = recall(sum_unique_prompt_train[0], sum_unique_prompt_train[2])
unique_f1_train = f1(unique_prec_train, unique_rec_train)

print('unique precision, recall and F1 score:', unique_prec_train, unique_rec_train, unique_f1_train)

composite_prec_train = precision(sum_composite_prompt_train[0], sum_composite_prompt_train[1])
composite_rec_train = recall(sum_composite_prompt_train[0], sum_composite_prompt_train[2])
composite_f1_train = f1(composite_prec_train, composite_rec_train)

print('composite precision, recall and F1 score:', composite_prec_train, composite_rec_train, composite_f1_train)

multivalued_prec_train = precision(sum_multivalued_prompt_train[0], sum_multivalued_prompt_train[1])
multivalued_rec_train = recall(sum_multivalued_prompt_train[0], sum_multivalued_prompt_train[2])
multivalued_f1_train = f1(multivalued_prec_train, multivalued_rec_train)

print('multivalued precision, recall and F1 score:', multivalued_prec_train, multivalued_rec_train, multivalued_f1_train)

relation_prec_train = precision(sum_relation_prompt_train[0], sum_relation_prompt_train[1])
relation_rec_train = recall(sum_relation_prompt_train[0], sum_relation_prompt_train[2])
relation_f1_train = f1(relation_prec_train, relation_rec_train)

print('relation precision, recall and F1 score:', relation_prec_train, relation_rec_train, relation_f1_train)

ent: 0.9324324324324325 0.8625 0.8961038961038962
attr: 0.8153153153153153 0.8701923076923077 0.841860465116279
unique: 0.8837209302325582 0.6229508196721312 0.7307692307692308
composite: 0.0 0 0
multivalued: 0.125 1.0 0.2222222222222222
relation: 0.7558139534883721 0.7471264367816092 0.7514450867052023


3. result of best combination with separated evaluations - test set

In [ ]:
# ent
ent = [[7,0,4], [3,0,6],[4,0,0],[5,0,0],[1,0,2],[4,0,0], [2,0,5],[4,0,0],[3,0,0],[3,0,1],[1,0,4]]
# attr
attr = [[19,11,0],[6,8,19],[5,1,10],[27,2,0],[0,1,0],[10,0,6],[9,1,13],[21,0,0],[10,0,0],[11,0,5],[0,2,0]]
# unique
unique = [[0,11,0],[4,0,0],[4,0,0],[3,0,2],[0,0,0],[3,0,1],[3,0,2],[4,0,0],[3,0,0],[3,0,1],[0,0,0]]
# composite
composite = [[0,2,0],[0,3,0],[0,4,0],[0,0,0],[0,1,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,1,0]]
# multivalued
multivalued = [[0,5,0],[0,7,0],[0,0,0],[0,0,0],[0,1,0],[0,2,0],[0,1,0],[0,1,0],[0,0,0],[0,0,0],[0,1,0]]
# relation
relation = [[8,5,3],[9,6,1],[3,2,1],[6,0,0],[3,0,0],[0,1,3],[0,1,4],[4,0,0],[2,0,0],[1,2,1],[2,0,2]]

In [ ]:
ent_np_prompt_test = np.array(ent)
attr_np_prompt_test = np.array(attr)
unique_np_prompt_test = np.array(unique)
composite_np_prompt_test = np.array(composite)
multivalued_np_prompt_test = np.array(multivalued)
relation_np_prompt_test = np.array(relation)

sum_ent_prompt_test = np.sum(ent_np_prompt_test, axis=0)
sum_attr_prompt_test = np.sum(attr_np_prompt_test, axis=0)
sum_unique_prompt_test = np.sum(unique_np_prompt_test, axis=0)
sum_composite_prompt_test = np.sum(composite_np_prompt_test, axis=0)
sum_multivalued_prompt_test = np.sum(multivalued_np_prompt_test, axis=0)
sum_relation_prompt_test = np.sum(relation_np_prompt_test, axis=0)

print('ent tp, fp and fn:',sum_ent_prompt_test)
print('attr tp, fp and fn:',sum_attr_prompt_test)
print('unique tp, fp and fn:',sum_unique_prompt_test)
print('composite tp, fp and fn:',sum_composite_prompt_test)
print('multivalued tp, fp and fn:',sum_multivalued_prompt_test)
print('relationships tp, fp and fn:',sum_relation_prompt_test)

ent: [37  0 22]
attr: [118  26  53]
unique: [27 11  6]
composite: [ 0 11  0]
multivalued: [ 0 18  0]
relationships: [38 17 15]


In [ ]:
ent_prec_test = precision(sum_ent_prompt_test[0], sum_ent_prompt_test[1])
ent_rec_test = recall(sum_ent_prompt_test[0], sum_ent_prompt_test[2])
ent_f1_test = f1(ent_prec_test, ent_rec_test)

print('ent precision, recall and F1 score:', ent_prec_test, ent_rec_test, ent_f1_test)

attr_prec_test = precision(sum_attr_prompt_test[0], sum_attr_prompt_test[1])
attr_rec_test = recall(sum_attr_prompt_test[0], sum_attr_prompt_test[2])
attr_f1_test = f1(attr_prec_test, attr_rec_test)

print('attr precision, recall and F1 score:', attr_prec_test, attr_rec_test, attr_f1_test)

unique_prec_test = precision(sum_unique_prompt_test[0], sum_unique_prompt_test[1])
unique_rec_test = recall(sum_unique_prompt_test[0], sum_unique_prompt_test[2])
unique_f1_test = f1(unique_prec_test, unique_rec_test)

print('unique precision, recall and F1 score:', unique_prec_test, unique_rec_test, unique_f1_test)

composite_prec_test = precision(sum_composite_prompt_test[0], sum_composite_prompt_test[1])
composite_rec_test = recall(sum_composite_prompt_test[0], sum_composite_prompt_test[2])
composite_f1_test = f1(composite_prec_test, composite_rec_test)

print('composite precision, recall and F1 score:', composite_prec_test, composite_rec_test, composite_f1_test)

multivalued_prec_test = precision(sum_multivalued_prompt_test[0], sum_multivalued_prompt_test[1])
multivalued_rec_test = recall(sum_multivalued_prompt_test[0], sum_multivalued_prompt_test[2])
multivalued_f1_test = f1(multivalued_prec_test, multivalued_rec_test)

print('multivalued precision, recall and F1 score:', multivalued_prec_test, multivalued_rec_test, multivalued_f1_test)

relation_prec_test = precision(sum_relation_prompt_test[0], sum_relation_prompt_test[1])
relation_rec_test = recall(sum_relation_prompt_test[0], sum_relation_prompt_test[2])
relation_f1_test = f1(relation_prec_test, relation_rec_test)

print('relation precision, recall and F1 score:', relation_prec_test, relation_rec_test, relation_f1_test)

ent: 1.0 0.6271186440677966 0.7708333333333333
attr: 0.8194444444444444 0.6900584795321637 0.7492063492063492
unique: 0.7105263157894737 0.8181818181818182 0.7605633802816901
composite: 0.0 0 0
multivalued: 0.0 0 0
relation: 0.6909090909090909 0.7169811320754716 0.7037037037037037
